In [1]:
!pip install torch torchaudio librosa soundfile scikit-learn tqdm
!pip install panns-inference

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import os
import random
import numpy as np
from glob import glob

import librosa
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Config
DATA_DIR = "/kaggle/input/emergency-vehicle-siren-sounds/sounds"
SR = 22050
N_MELS = 64
N_FFT = 1024
HOP_LENGTH = 512
DURATION = 4.0
SAMPLES = int(SR * DURATION)
BATCH_SIZE = 32
LR = 1e-3
EPOCHS = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42
MODEL_PATH = "/kaggle/working/rcnn_siren_best.pt"

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# Dataset utilities
def load_audio(path):
    y, _ = librosa.load(path, sr=SR, mono=True)
    if len(y) < SAMPLES:
        y = np.pad(y, (0, SAMPLES - len(y)))
    else:
        y = y[:SAMPLES]
    return y

def compute_mel(y):
    mel = librosa.feature.melspectrogram(
        y=y,
        sr=SR,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        n_mels=N_MELS
    )
    log_mel = librosa.power_to_db(mel, ref=np.max)
    log_mel = (log_mel - log_mel.mean()) / (log_mel.std() + 1e-9)
    return log_mel

class SirenDataset(Dataset):
    def __init__(self, files, labels):
        self.files = files
        self.labels = labels

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        y = load_audio(self.files[idx])
        m = compute_mel(y)
        x = np.expand_dims(m, axis=0)
        return torch.tensor(x), torch.tensor(self.labels[idx], dtype=torch.long)

# Gather files
classes = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
filepaths, labels = [], []
for c in classes:
    for ext in ("*.wav", "*.mp3"):
        for f in glob(os.path.join(DATA_DIR, c, ext)):
            filepaths.append(f)
            labels.append(c)

# Encode labels
le = LabelEncoder()
labels = le.fit_transform(labels)

from sklearn.model_selection import train_test_split

# === Split into train, val, test ===
X_temp, X_test, y_temp, y_test = train_test_split(
    filepaths, labels, test_size=0.1, stratify=labels, random_state=SEED
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, stratify=y_temp, random_state=SEED
)
# Result: 72% train, 18% val, 10% test

# === Datasets ===
train_ds = SirenDataset(X_train, y_train)
val_ds = SirenDataset(X_val, y_val)
test_ds = SirenDataset(X_test, y_test)

# === DataLoaders ===
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)


# RCNN model
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1), nn.BatchNorm2d(out_ch), nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1), nn.BatchNorm2d(out_ch), nn.ReLU(),
            nn.MaxPool2d(2)
        )
    def forward(self, x): return self.net(x)

class RCNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.c1 = ConvBlock(1, 32)
        self.c2 = ConvBlock(32, 64)
        self.c3 = ConvBlock(64, 128)
        self.rnn_input = 128 * (N_MELS // 8)
        self.rnn = nn.GRU(self.rnn_input, 128, num_layers=2, batch_first=True, bidirectional=True, dropout=0.3)
        self.fc = nn.Sequential(nn.Linear(256, 128), nn.ReLU(), nn.Dropout(0.3), nn.Linear(128, n_classes))

    def forward(self, x):
        x = self.c1(x)
        x = self.c2(x)
        x = self.c3(x)
        B, C, M, T = x.shape
        x = x.permute(0, 3, 1, 2).contiguous().view(B, T, C*M)
        x, _ = self.rnn(x)
        x = x.mean(1)
        return self.fc(x)

model = RCNN(len(le.classes_)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Train loop
best_acc = 0
for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss, correct, total = 0, 0, 0
    for xb, yb in tqdm(train_loader, desc=f"Epoch {epoch}"):
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
        correct += (preds.argmax(1) == yb).sum().item()
        total += xb.size(0)
    train_acc = correct / total

    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            preds = model(xb)
            loss = criterion(preds, yb)
            val_loss += loss.item() * xb.size(0)
            correct += (preds.argmax(1) == yb).sum().item()
            total += xb.size(0)
    val_acc = correct / total

    print(f"Epoch {epoch} | Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model, MODEL_PATH)
        print(f"✅ Saved best model at {MODEL_PATH}")

print(f"Best Val Accuracy: {best_acc:.3f}")


Epoch 1: 100%|██████████| 14/14 [00:30<00:00,  2.21s/it]


Epoch 1 | Train Acc: 0.736 | Val Acc: 0.713
✅ Saved best model at /kaggle/working/rcnn_siren_best.pt


Epoch 2: 100%|██████████| 14/14 [00:06<00:00,  2.23it/s]


Epoch 2 | Train Acc: 0.907 | Val Acc: 0.880
✅ Saved best model at /kaggle/working/rcnn_siren_best.pt


Epoch 3: 100%|██████████| 14/14 [00:05<00:00,  2.36it/s]


...

Epoch 98 | Train Acc: 1.000 | Val Acc: 0.991


Epoch 99: 100%|██████████| 14/14 [00:05<00:00,  2.47it/s]


Epoch 99 | Train Acc: 1.000 | Val Acc: 0.991


Epoch 100: 100%|██████████| 14/14 [00:06<00:00,  2.31it/s]


Epoch 100 | Train Acc: 1.000 | Val Acc: 0.991
Best Val Accuracy: 0.991


In [3]:
# ===== Load the best model =====
device = torch.device("cuda")
best_model_path = "/kaggle/working/rcnn_siren_best.pt"  # adjust path if needed
model = torch.load(best_model_path, map_location=device, weights_only=False)
model.to(device)
model.eval()

# ===== Evaluation =====
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(targets.cpu().numpy())

# ===== Metrics =====
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, digits=4))

print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

# ===== Accuracy =====
accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"Test Accuracy: {accuracy:.4f}")


Classification Report:
              precision    recall  f1-score   support

           0     0.9048    0.9500    0.9268        20
           1     0.9474    0.9000    0.9231        20
           2     1.0000    1.0000    1.0000        20

    accuracy                         0.9500        60
   macro avg     0.9507    0.9500    0.9500        60
weighted avg     0.9507    0.9500    0.9500        60

Confusion Matrix:
[[19  1  0]
 [ 2 18  0]
 [ 0  0 20]]
Test Accuracy: 0.9500
